In [70]:
"""
1.	Нужно чтобы программа обновляла данные в столбцах которые есть в первой таблице при совпадении идентификатора.
2.	Добавляла данные в соответствующие столбцы если такого идентификатора нет в первой таблице
Скорее всего нужно создавать дополнительный лист в главном файле и сохранять его отдельно, так будет проще
"""
import pandas as pd
import numpy as np
import os

import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from openpyxl import load_workbook
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk
import time
import datetime
import warnings
from dateutil.parser import ParserError

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
pd.options.mode.chained_assignment = None
import sys
import locale
import logging
import tempfile

In [10]:
def convert_params_columns_to_int(lst):
    """
    Функция для конвератации значений колонок которые нужно обработать.
    Очищает от пустых строк, чтобы в итоге остался список из чисел в формате int
    """
    out_lst = [] # Создаем список в который будем добавлять только числа
    for value in lst: # Перебираем список
        try:
            # Обрабатываем случай с нулем, для того чтобы после приведения к питоновскому отсчету от нуля не получилась колонка с номером -1
            number = int(value)
            if number != 0:
                out_lst.append(value) # Если конвертирования прошло без ошибок то добавляем
            else:
                continue
        except: # Иначе пропускаем
            continue
    return out_lst


def convert_columns_to_str(df, number_columns):
    """
    Функция для конвертации указанных столбцов в строковый тип и очистки от пробельных символов в начале и конце
    """

    for column in number_columns:  # Перебираем список нужных колонок
        try:
            df.iloc[:, column] = df.iloc[:, column].astype(str)
            # Очищаем колонку от пробельных символов с начала и конца
            df.iloc[:, column] = df.iloc[:, column].apply(lambda x: x.strip())
        except IndexError:
            messagebox.showerror('Веста Обработка таблиц и создание документов ver 1.21',
                                 'Проверьте порядковые номера колонок которые вы хотите обработать.')
            
def processing_date_column(df, lst_columns):
    """
    Функция для обработки столбцов с датами. конвертация в строку формата ДД.ММ.ГГГГ
    """
    # получаем первую строку
    first_row = df.iloc[0, lst_columns]

    lst_first_row = list(first_row)  # Превращаем строку в список
    lst_date_columns = []  # Создаем список куда будем сохранять колонки в которых находятся даты
    tupl_row = list(zip(lst_columns,
                        lst_first_row))  # Создаем список кортежей формата (номер колонки,значение строки в этой колонке)

    for idx, value in tupl_row:  # Перебираем кортеж
        result = check_date_columns(idx, value)  # проверяем является ли значение датой
        if result:  # если да то добавляем список порядковый номер колонки
            lst_date_columns.append(result)
        else:  # иначе проверяем следующее значение
            continue
    for i in lst_date_columns:  # Перебираем список с колонками дат, превращаем их в даты и конвертируем в нужный строковый формат
        df.iloc[:, i] = pd.to_datetime(df.iloc[:, i], errors='coerce', dayfirst=True)
        df.iloc[:, i] = df.iloc[:, i].apply(create_doc_convert_date)

def check_date_columns(i, value):
    """
    Функция для проверки типа колонки. Необходимо найти колонки с датой
    :param i:
    :param value:
    :return:
    """
    try:
        itog = pd.to_datetime(str(value), infer_datetime_format=True)

    except ParserError:
        pass
    except ValueError:
        pass
    except TypeError:
        pass
    else:
        return i

def create_doc_convert_date(cell):
    """
    Функция для конвертации даты при создании документов
    :param cell:
    :return:
    """
    try:
        string_date = datetime.datetime.strftime(cell, '%d.%m.%Y')
        return string_date
    except ValueError:
        return 'Не удалось конвертировать дату.Проверьте значение ячейки!!!'
    except TypeError:
        return 'Не удалось конвертировать дату.Проверьте значение ячейки!!!'    
    
    


In [66]:
def update_value(orig_value,new_value,value_filter):
    """
    Функция обновляет данные в колонке зависимости от значения в колонке фильтре   
    """
    if value_filter == 'both':
        return new_value
    else:
        return orig_value
    

In [ ]:
# Получаем значения текстовых полей
# first_sheet_name = str(entry_first_sheet_name.get())
# second_sheet_name = str(entry_second_sheet_name.get())

# # загружаем файлы
# first_df = pd.read_excel(name_first_file_comparison, sheet_name=first_sheet_name, dtype=str,
#                          keep_default_na=False)
# # получаем имя файла
# name_first_df = name_first_file_comparison.split('/')[-1]
# name_first_df = name_first_df.split('.xlsx')[0]

# second_df = pd.read_excel(name_second_file_comparison, sheet_name=second_sheet_name, dtype=str,
#                           keep_default_na=False)
# # получаем имя файла
# name_second_df = name_second_file_comparison.split('/')[-1]
# name_second_df = name_second_df.split('.xlsx')[0]

# params = pd.read_excel(file_params, header=None, keep_default_na=False)





In [53]:
first_sheet_name = 'Worksheet'
second_sheet_name = 'Лист1'
name_first_file_comparison = 'data/Основная таблица.xlsx'
name_second_file_comparison = 'data/Социалка.xlsx'
file_params = 'data/Параметры слияния.xlsx'

first_df = pd.read_excel(name_first_file_comparison, sheet_name=first_sheet_name, dtype=str,
                         keep_default_na=False)
# получаем имя файла
name_first_df = name_first_file_comparison.split('/')[-1]
name_first_df = name_first_df.split('.xlsx')[0]

second_df = pd.read_excel(name_second_file_comparison, sheet_name=second_sheet_name, dtype=str,
                          keep_default_na=False)
# получаем имя файла
name_second_df = name_second_file_comparison.split('/')[-1]
name_second_df = name_second_df.split('.xlsx')[0]

params = pd.read_excel(file_params, header=None, keep_default_na=False)


In [54]:

# Преврашаем каждую колонку в список
params_first_columns = params[0].tolist()
params_second_columns = params[1].tolist()

# Конвертируем в инт заодно проверяя корректность введенных данных
int_params_first_columns = convert_params_columns_to_int(params_first_columns)
int_params_second_columns = convert_params_columns_to_int(params_second_columns)

# Отнимаем 1 от каждого значения чтобы привести к питоновским индексам
int_params_first_columns = list(map(lambda x: x - 1, int_params_first_columns))
int_params_second_columns = list(map(lambda x: x - 1, int_params_second_columns))

# Конвертируем нужные нам колонки в str
convert_columns_to_str(first_df, int_params_first_columns)
convert_columns_to_str(second_df, int_params_second_columns)

# в этом месте конвертируем даты в формат ДД.ММ.ГГГГ
processing_date_column(first_df, int_params_first_columns)
processing_date_column(second_df, int_params_second_columns)

In [55]:
# Проверяем наличие колонки _merge
if '_merge' in first_df.columns:
    first_df.drop(columns=['_merge'], inplace=True)
if '_merge' in second_df.columns:
    second_df.drop(columns=['_merge'], inplace=True)
# Проверяем наличие колонки ID
if 'ID_объединения' in first_df.columns:
    first_df.drop(columns=['ID_объединения'], inplace=True)
if 'ID_объединения' in second_df.columns:
    second_df.drop(columns=['ID_объединения'], inplace=True)

In [56]:
first_df.head(1)

,Фамилия,Имя,Отчество,ФИО,Дата рождения,Номер телефона,Электронная почта,Адрес прописки,Фактический адрес,Серия и номер паспорта,Серия паспорта,Номер паспорта,Код подразделения,Кем выдан,Дата выдачи паспорта,ИНН ФЗ/ИП,СНИЛС,ОМС,Направление
0,Алехин,Данила,Прокопьевич,Алехин Данила Прокопьевич,14.04.1973,+7 (951) 885-25-41,danila79@hotmail.com,"Россия, г. Иркутск, Кирова ул., д. 1 кв.212","Россия, г. Екатеринбург, Кирова ул., д. 1 кв.107",4844 295530,4844,295530,330-554,ОУФМС России по г. Екатеринбург,08.08.2020,158493217864,707-920-812 02,5950614844448865,02.03.03 Математическое обеспечение и админист...


In [57]:
second_df.head(1)

,ФИО,Номер телефона,Фактический адрес,Серия паспорта,Номер паспорта,СНИЛС,Статус,Подтверждающий документ,Серия документа,Номер документа
0,Байдавлетова Виктория Юрьевна,+7 (940) 710-41-74,"Россия, г. Дербент, Красноармейская ул., д. 12...",4317,888888,409-535-029 76,ОВЗ,Справка,АЭС730,70172


In [58]:


# Создаем в каждом датафрейме колонку с айди путем склеивания всех нужных колонок в одну строку
first_df['ID_объединения'] = first_df.iloc[:, int_params_first_columns].sum(axis=1)
second_df['ID_объединения'] = second_df.iloc[:, int_params_second_columns].sum(axis=1)
# Удаляем все пробелы чтобы они не повлияли на слияние
first_df['ID_объединения'] = first_df['ID_объединения'].apply(lambda x: x.replace(' ', ''))
second_df['ID_объединения'] = second_df['ID_объединения'].apply(lambda x: x.replace(' ', ''))



In [59]:

# Обрабатываем дубликаты

duplicates_first_df = first_df[first_df.duplicated(subset=['ID_объединения'],
                                                   keep=False)]  # Сохраняем все значения у которых есть дубликаты в отдельный датафрейм

first_df.drop_duplicates(subset=['ID_объединения'], keep=False, inplace=True)  # Удаляем дубликаты из датафрейма

duplicates_second_df = second_df[second_df.duplicated(subset=['ID_объединения'],
                                                      keep=False)]  # Сохраняем все значения у которых есть дубликаты в отдельный датафрейм
second_df.drop_duplicates(subset=['ID_объединения'], keep=False, inplace=True)  # Удаляем дубликаты из датафрейма

# # Проверяем размер датафрейма с дубликатами, если он больше 0 то выдаем сообшение пользователю
if duplicates_first_df.shape[0] > 0:
    messagebox.showwarning('Веста Обработка таблиц и создание документов ver 1.21',
                           f'В первой таблице обнаружены дубликаты!!!\nДля корректного объединения таблиц ,дубликаты перенесены в отдельный лист итоговой таблицы')
if duplicates_second_df.shape[0] > 0:
    messagebox.showwarning('Веста Обработка таблиц и создание документов ver 1.21',
                           f'Во второй таблице обнаружены дубликаты!!!\nДля корректного объединения таблиц ,дубликаты перенесены в отдельный лист итоговой таблицы')

In [60]:
# В результат объединения попадают совпадающие по ключу записи обеих таблиц и все строки из этих двух таблиц, для которых пар не нашлось. Порядок таблиц в запросе не

# Создаем документ
wb = openpyxl.Workbook()
# создаем листы
ren_sheet = wb['Sheet']
ren_sheet.title = 'Таблица 1'
wb.create_sheet(title='Таблица 2', index=1)
wb.create_sheet(title='Совпадающие данные', index=2)
wb.create_sheet(title='Обновленная таблица', index=3)
wb.create_sheet(title='Объединённая таблица', index=4)
# Создаем листы для дубликатов
wb.create_sheet(title='Дубликаты первая таблица', index=5)
wb.create_sheet(title='Дубликаты вторая таблица', index=6)

<Worksheet "Дубликаты вторая таблица">

In [61]:

# Проводим слияние
itog_df = pd.merge(first_df, second_df, how='outer', left_on=['ID_объединения'], right_on=['ID_объединения'],
                   indicator=True)

In [72]:
# копируем в отдельный датафрейм чтобы 
update_df = itog_df.copy()

# получаем список с совпадающими колонками первой таблицы
first_df_columns = [column for column in list(update_df.columns) if str(column).endswith('_x')]
# получаем список с совпадающими колонками второй таблицы
second_df_columns = [column for column in list(update_df.columns) if str(column).endswith('_y')]
# Создаем из списка совпадающих колонок второй таблицы словарь, чтобы было легче обрабатывать
# да конечно можно было сделать в одном выражении но как я буду читать это через 2 недели?
dct_second_columns = {column.split('_y')[0]:column for column in second_df_columns}

In [73]:
dct_second_columns

{'ФИО': 'ФИО_y',
 'Номер телефона': 'Номер телефона_y',
 'Фактический адрес': 'Фактический адрес_y',
 'Серия паспорта': 'Серия паспорта_y',
 'Номер паспорта': 'Номер паспорта_y',
 'СНИЛС': 'СНИЛС_y'}

In [74]:
for column in first_df_columns:
    # очищаем от _x
    name_column = column.split('_x')[0]
    update_df[column] = np.where(update_df['_merge']=='both',update_df[dct_second_columns[name_column]],update_df[column]) 
   

    
    
    

In [ ]:
#     update_df[column] = update_df.apply(lambda x:update_value(update_df[column],update_df[dct_second_columns[name_column]],
#                                                              update_df['_merge']),axis=1)
    

In [76]:
update_df.to_excel('update.xlsx',index=False)

In [ ]:


# Создаем переменные содержащие в себе количество колонок в базовых датареймах
first_df_quantity_cols = len(first_df.columns)  # не забываем что там добавилась колонка ID

# Записываем каждый датафрейм в соответсвующий лист
# Левая таблица
left_df = itog_df[itog_df['_merge'] == 'left_only']
left_df.drop(['_merge'], axis=1, inplace=True)

# Удаляем колонки второй таблицы чтобы не мешались
left_df.drop(left_df.iloc[:, first_df_quantity_cols:], axis=1, inplace=True)

# Переименовываем колонки у которых были совпадение во второй таблице, в таких колонках есть добавление _x
clean_left_columns = list(map(lambda x: x[:-2] if '_x' in x else x, list(left_df.columns)))
left_df.columns = clean_left_columns
for r in dataframe_to_rows(left_df, index=False, header=True):
    wb['Таблица 1'].append(r)

right_df = itog_df[itog_df['_merge'] == 'right_only']
right_df.drop(['_merge'], axis=1, inplace=True)

# Удаляем колонки первой таблицы таблицы чтобы не мешались
right_df.drop(right_df.iloc[:, :first_df_quantity_cols - 1], axis=1, inplace=True)

# Переименовываем колонки у которых были совпадение во второй таблице, в таких колонках есть добавление _x
clean_right_columns = list(map(lambda x: x[:-2] if '_y' in x else x, list(right_df.columns)))
right_df.columns = clean_right_columns

for r in dataframe_to_rows(right_df, index=False, header=True):
    wb['Таблица 2'].append(r)

both_df = itog_df[itog_df['_merge'] == 'both']
both_df.drop(['_merge'], axis=1, inplace=True)
# Очищаем от _x  и _y
clean_both_columns = clean_ending_columns(list(both_df.columns), name_first_df, name_second_df)
both_df.columns = clean_both_columns

for r in dataframe_to_rows(both_df, index=False, header=True):
    wb['Совпадающие данные'].append(r)

# Сохраняем общую таблицу
# Заменяем названия индикаторов на более понятные
itog_df['_merge'] = itog_df['_merge'].apply(lambda x: 'Данные из первой таблицы' if x == 'left_only' else
('Данные из второй таблицы' if x == 'right_only' else 'Совпадающие данные'))
itog_df['_merge'] = itog_df['_merge'].astype(str)

clean_itog_df = clean_ending_columns(list(itog_df.columns), name_first_df, name_second_df)
itog_df.columns = clean_itog_df
for r in dataframe_to_rows(itog_df, index=False, header=True):
    wb['Объединённая таблица'].append(r)

# Записываем дубликаты в соответствующие листы
for r in dataframe_to_rows(duplicates_first_df, index=False, header=True):
    wb['Дубликаты первая таблица'].append(r)

for r in dataframe_to_rows(duplicates_second_df, index=False, header=True):
    wb['Дубликаты вторая таблица'].append(r)
# генерируем текущее время
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
# Сохраняем итоговый файл
wb.save(f'{path_to_end_folder_comparison}/Результат слияния 2 таблиц от {current_time}.xlsx')